In [ ]:
import zipfile

def unzip_folder(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

# Example usage:
zip_path = '/content/VisDrone2019-DET-val.zip'
extract_to = '/content/'
unzip_folder(zip_path, extract_to)


In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kushagrapandya/visdrone-dataset

Dataset URL: https://www.kaggle.com/datasets/kushagrapandya/visdrone-dataset
License(s): GNU Affero General Public License 3.0
100% 2.09G/2.10G [01:56<00:00, 16.7MB/s]
100% 2.10G/2.10G [01:56<00:00, 19.4MB/s]


In [ ]:
!unzip visdrone-dataset.zip -d visdrone-dataset

Streaming output truncated to the last 5000 lines.
  inflating: visdrone-dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/images/9999951_00000_d_0000235.jpg  
  inflating: visdrone-dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/images/9999951_00000_d_0000236.jpg  
  inflating: visdrone-dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/images/9999951_00000_d_0000237.jpg  
  inflating: visdrone-dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/images/9999951_00000_d_0000238.jpg  
  inflating: visdrone-dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/images/9999951_00000_d_0000239.jpg  
  inflating: visdrone-dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/images/9999951_00000_d_0000240.jpg  
  inflating: visdrone-dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/images/9999951_00000_d_0000241.jpg  
  inflating: visdrone-dataset/VisDrone2019-DET-train/VisDrone2019-DET-train/images/9999951_00000_d_0000242.jpg  
  inflating: visdrone-dataset/VisDrone2019-DE

In [ ]:
!pip install ultralytics
!pip install -U matplotlib  # Optional for better visualization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 101.4 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.8.0
    Uninstalling matplotlib-3.8.0:
      Successfully uninstalled matplotlib-3.8.0


In [ ]:
import os
from pathlib import Path
from PIL import Image
from tqdm import tqdm

# Define the dataset path
dataset_path = '/content/visdrone-dataset/VisDrone2019-DET-val/VisDrone2019-DET-val'
train_images = os.path.join(dataset_path, 'images')
val_images = os.path.join(dataset_path, 'images')
test_images = os.path.join(dataset_path, 'images')

# Download and convert VisDrone dataset to YOLO format
def visdrone2yolo(dir):
    def convert_box(size, box):
        dw = 1. / size[0]
        dh = 1. / size[1]
        return (box[0] + box[2] / 2) * dw, (box[1] + box[3] / 2) * dh, box[2] * dw, box[3] * dh

    # Ensure the labels directory exists
    (dir / 'labels').mkdir(parents=True, exist_ok=True)

    # Process all annotation files
    pbar = tqdm((dir / 'annotations').glob('*.txt'), desc=f'Converting {dir}')

    for f in pbar:
        img_file = (dir / 'images' / f.name).with_suffix('.jpg')

        # Only process if the image exists
        if img_file.exists():
            img_size = Image.open(img_file).size
            lines = []
            with open(f, 'r') as file:
                for row in [x.split(',') for x in file.read().strip().splitlines()]:
                    if row[4] == '0':  # Skip ignored regions (class 0)
                        continue
                    cls = int(row[5]) - 1
                    box = convert_box(img_size, tuple(map(int, row[:4])))
                    lines.append(f"{cls} {' '.join(f'{x:.6f}' for x in box)}\n")

            # Write all lines to the corresponding label file at once
            label_file = str(f).replace(f'{os.sep}annotations{os.sep}', f'{os.sep}labels{os.sep}')
            with open(label_file, 'w') as fl:
                fl.writelines(lines)

# Convert dataset directories
visdrone2yolo(Path(dataset_path))


Converting /content/visdrone-dataset/VisDrone2019-DET-val/VisDrone2019-DET-val: 548it [00:00, 1299.15it/s]


In [ ]:
from ultralytics import YOLO
import os

# Load the pretrained YOLOv11s model
model = YOLO('yolo11s.pt')  # Use other models like yolov5m.pt, yolov5l.pt, or yolov5x.pt for larger models

# Verify the path to the dataset YAML file
print(os.listdir('/content/visdrone-dataset/VisDrone2019-DET-train/VisDrone2019-DET-train'))

# Train the model, save weights every 5 epochs
model.train(data='/content/visdrone-dataset/VisDrone.yaml', epochs=100, batch=16, imgsz=640, pretrained=True, save_period=5)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 18.4M/18.4M [00:00<00:00, 290MB/s]


['annotations', 'images']
Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/content/visdrone-dataset/VisDrone.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_

Converting /content/datasets/VisDrone/VisDrone2019-DET-train: 6471it [00:42, 151.09it/s]
Converting /content/datasets/VisDrone/VisDrone2019-DET-val: 548it [00:05, 94.25it/s]
Converting /content/datasets/VisDrone/VisDrone2019-DET-test-dev: 1610it [00:09, 177.24it/s]

Dataset download success ✅ (150.2s), saved to /content/datasets



...

     56/100      9.24G      1.208     0.8063     0.8836        654        640: 100%|██████████| 405/405 [03:22<00:00,  2.00it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:08<00:00,  2.18it/s]


                   all        548      38759      0.516       0.38      0.399       0.24

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     57/100      13.4G      1.217     0.8101     0.8848        684        640: 100%|██████████| 405/405 [03:25<00:00,  1.97it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:07<00:00,  2.49it/s]


                   all        548      38759      0.525      0.381        0.4       0.24

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     58/100      9.51G      1.203     0.8001     0.8812       1547        640:  78%|███████▊  | 317/405 [02:38<00:36,  2.41it/s]